In [27]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import time
import dask.bag as db
import pickle5 as pickle
from dask.diagnostics import ProgressBar


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# df = pd.read_csv('../data/Bio_entities_Main.csv', blocksize=16 * 1024 * 1024)
from dask.diagnostics import ProgressBar
import dask.bag as db

In [38]:
def create_entities_dict(chunk):
    """

        colums are       PMID       Start  End  EntityID
    chunk is :

array([[    30941,         0,         7,   4167203],
       [    30941,       333,       335, 292621503],
       [    30941,       344,       365, 271302003],
       [    30941,       625,       632,   4167203],
    """
    PMID=0
    START=1
    END = 2
    ENTITYID=3
    doc_entity_count = {}
    for i in range(len(chunk)):
        # pbar.update(1)
        # print(row)
        Pmid = chunk[i][PMID] #int(row.PMID)
        EntityID = chunk[i][START] #int(row.EntityID)
        Start = chunk[i][END] #int(row.Start)
        End = chunk[i][ENTITYID] # int(row.End)
        #ignore mentions with no entityID
        if EntityID == 0 or EntityID =='0':
            continue
        #new pmid        
        if Pmid not in doc_entity_count:    
            #init mention
            doc_entity_count[Pmid] = {EntityID: [[Start],[End]] }
            continue
        #
        # PMID exits in catalog but the entityID found is new    
        if EntityID not in doc_entity_count[Pmid]:
            doc_entity_count[Pmid][EntityID]={}
            # doc_entity_count[Pmid][EntityID]['count'] = 1
            doc_entity_count[Pmid][EntityID] = [[Start], [End]]
        else:#Increase counter
            # doc_entity_count[Pmid][EntityID]['count'] += 1
            doc_entity_count[Pmid][EntityID][0].append(Start)
            doc_entity_count[Pmid][EntityID][1].append(End)
    
    return doc_entity_count

In [39]:
df = pd.read_csv('../data/Bio_entities_small.csv',usecols=['PMID','Start','End','EntityID'])
df = df[df.EntityID != 0]
data = df.to_numpy()


In [40]:
%%time
tmp_d = create_entities_dict(data)

CPU times: user 763 ms, sys: 71.9 ms, total: 835 ms
Wall time: 837 ms


In [49]:
tmp_d[30941]

{18: [[25], [4631803]],
 31: [[40], [44532203]],
 50: [[57], [4631803]],
 59: [[68], [44532203]],
 281: [[288], [4631803]],
 407: [[416], [44532203]],
 538: [[545], [4631803]],
 558: [[567], [44532203]],
 640: [[647], [4631803]],
 685: [[692], [4631803]],
 719: [[728], [44532203]],
 850: [[857], [4631803]],
 925: [[932], [4631803]],
 999: [[1006], [4631803]],
 1241: [[1248], [4631803]],
 1319: [[1328], [44532203]],
 1439: [[1446], [4631803]],
 1450: [[1459], [44532203]],
 1495: [[1504], [44532203]],
 1546: [[1553], [4631803]],
 1582: [[1589], [4631803]],
 1600: [[1609], [44532203]],
 1698: [[1707], [44532203]],
 1713: [[1720], [4631803]],
 1831: [[1840], [44532203]],
 1850: [[1857], [4631803]],
 225: [[238], [840105]]}

In [53]:
%%time
results = {}
CHUNK_SIZE =50000000 
for df in pd.read_csv('../data/Bio_entities_Main.csv',usecols=['Start','End', 'EntityID', 'PMID'], iterator=True, chunksize=CHUNK_SIZE):

# for df in pd.read_csv( '../data/Bio_entities_small.csv', usecols=['Start','End', 'EntityID', 'PMID'], iterator=True ):

    df = df[df.EntityID != 0]
    data = df.to_numpy()
    print(data.shape)
    # print(data[:10])

    _CHUNK_SIZE = int(data.shape[0]/100)
    chunks = np.array_split(data, _CHUNK_SIZE, axis=0)
    len(chunks)
    del data

    mybag = db.from_sequence(chunks)
    with ProgressBar():
        myresult = mybag.map(create_entities_dict).compute()

    # import pdb; pdb.set_trace()
    print(len(myresult))
    #flatten list
    result = {}
    for lis in myresult:
        result.update(lis)

    results.update(result)
    # print(len(result))
    # results.append(myresult)

(40301971, 4)
[########################################] | 100% Completed |  5min 51.6s
403019
(39763503, 4)
[########################################] | 100% Completed |  6min 24.8s
397635
(40035951, 4)
[########################################] | 100% Completed |  7min 12.1s
400359
(40125227, 4)
[########################################] | 100% Completed |  5min 30.2s
401252
(40132144, 4)
[########################################] | 100% Completed |  6min 30.1s
401321
(40778145, 4)
[########################################] | 100% Completed |  7min 48.3s
407781
(24167223, 4)
[########################################] | 100% Completed |  9min  3.1s
241672
CPU times: user 47min 21s, sys: 14min 44s, total: 1h 2min 6s
Wall time: 1h 2min 19s


In [55]:
results[30941]

{333: [[335], [292621503]],
 344: [[365], [271302003]],
 625: [[632], [4167203]],
 810: [[813], [314219703]],
 833: [[834], [301100803]],
 843: [[850], [4167203]],
 1080: [[1087], [4167203]],
 1158: [[1159], [301100803]],
 1195: [[1196], [301100803]],
 446: [[449], [323589602]],
 722: [[725], [323589602]],
 763: [[764], [326403902]],
 40: [[44], [960605]],
 146: [[150], [960605]],
 190: [[194], [960605]]}

In [51]:
#Seems that something is fishy
#results == tmp_d

False

In [8]:
import bz2

In [56]:
handle = bz2.BZ2File('pmid2entities.bz2', 'w')
pickle.dump(results, handle,
                protocol = pickle.HIGHEST_PROTOCOL)

In [57]:
%%time
handle = bz2.BZ2File('pmid2entities.bz2','r')
pmid2entities = pickle.load(handle)

CPU times: user 25min 1s, sys: 3min 37s, total: 28min 38s
Wall time: 28min 37s


In [11]:
pmid2entities[30941]

{4167203: [[0, 625, 843, 1080], [7, 632, 850, 1087]],
 292621503: [[333], [335]],
 271302003: [[344], [365]],
 314219703: [[810], [813]],
 301100803: [[833, 1158, 1195], [834, 1159, 1196]],
 323589602: [[446, 722], [449, 725]],
 326403902: [[763], [764]],
 960605: [[40, 146, 190], [44, 150, 194]]}